Library Imports


In [1]:
import cv2
import numpy as np

2 Frames Alignment Function 


In [2]:
# This script aligns two images using feature-based methods.
# It uses ORB feature detection and matching to compute a homography matrix,   
# which is then used to warp one image to align with the other.
# The aligned image is then tested against the original image to ensure alignment.
img1 = cv2.imread('cam0_me.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('cam1_me.jpg', cv2.IMREAD_GRAYSCALE)

def align_images_feature_based(img1, img2):
    # Convert both to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    if len(matches) < 4:
        raise ValueError("Not enough matches to compute homography.")

    matches = sorted(matches, key=lambda x: x.distance)

    # Extract location of good matches
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate the homography matrix
    M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    # Warp img2 to align with img1
    aligned = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))
    return aligned

def overlay_images_rgb(img1, img2_aligned, alpha=0.5):
    # Resize img2_aligned to match img1 if needed
    if img1.shape != img2_aligned.shape:
        img2_aligned = cv2.resize(img2_aligned, (img1.shape[1], img1.shape[0]))

    # Blend images using alpha
    blended = cv2.addWeighted(img1, alpha, img2_aligned, 1 - alpha, 0)
    return blended


# Load original and rotated images
img1 = cv2.imread("cam0_me.jpg")
img2 = cv2.imread("cam1_me.jpg")

# Align img2 to img1
aligned = align_images_feature_based(img1, img2)

# Overlay for visual verification
blended = overlay_images_rgb(img1, aligned)

# Save and show
cv2.imwrite("aligned_output_me.jpg", aligned)
cv2.imwrite("overlay_check_me.jpg", blended)


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


2 Frames Focus Assessment Function 

In [ ]:
def focus_loss(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    variance = laplacian.var()
    # Loss is high when image is blurry (i.e., variance is low)
    return -variance
loss1 = focus_loss(img1)
loss2 = focus_loss(img2)
print(f"Focus loss for img1: {loss1}, img2: {loss2}")
focus_threshold = 0.5  # Example threshold for focus quality
if np.abs(loss1 - loss2) < focus_threshold:
    print("The images are in focus.")
else:
    print("The images are not in focus.")

Focus loss for img1: -7.976629533076443, img2: -7.877956629389179
The images are in focus.


: 

Video Alignment Function


In [ ]:

# Open both videos
cap1 = cv2.VideoCapture("video1.mp4")  # reference
cap2 = cv2.VideoCapture("video2.mp4")  # to align

# Read one frame to get size/fps info
ret1, frame_ref = cap1.read()
ret2, frame_target = cap2.read()
if not (ret1 and ret2):
    raise ValueError("Failed to read the first frames.")

# Get video properties
h, w = frame_ref.shape[:2]
fps = cap1.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("video2_aligned.mp4", fourcc, fps, (w, h))

# Reset videos to the start
cap1.set(cv2.CAP_PROP_POS_FRAMES, 0)
cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Frame-by-frame alignment
while True:
    ret1, frame_ref = cap1.read()
    ret2, frame_target = cap2.read()
    if not (ret1 and ret2):
        break

    aligned_frame = align_images_feature_based(frame_ref, frame_target)
    out.write(aligned_frame)

cap1.release()
cap2.release()
out.release()


ValueError: Failed to read the first frames.